<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/twelvelabs-io/twelvelabs-developer-experience/blob/main/quickstarts/1.0.0-beta/TwelveLabs_Quickstart_Embeddings.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in  Colab</a>
  </td>
</table>

# Create embeddings

This guide shows how to utilize the TwelveLabs Python SDK to create embeddings for your videos.

# Prerequisites
Before you begin, ensure the following prerequisites are met:

- [Sign up](https://playground.twelvelabs.io/) for a free account and obtain your API key from the [API Key](https://playground.twelvelabs.io/dashboard/api-key) page. No credit card is required to use the Free plan. This plan allows you to index up to 600 minutes of videos, which is sufficient for a small project. 
- The videos you wish to upload must meet the requirements int the [Prerequisites](https://docs.twelvelabs.io/v1.3/docs/guides/create-embeddings/video/new#prerequisites) section of the **Create video embeddings** page.



# Procedure

## Install the TwelveLabs Python SDK

In [ ]:
%pip install twelvelabs

## Configure your API key


In [ ]:
# For Google Colab, store your API key as a Secret named `TL_API_KEY`. If you don't know how to create a Colab Secret, see https://medium.com/@parthdasawant/how-to-use-secrets-in-google-colab-450c38e3ec75.

from google.colab import userdata
TL_API_KEY = userdata.get('TL_API_KEY')

# For other Python environments, you can use environment variables
# TL_API_KEY = os.environ.get('TL_API_KEY')

## **Generate Embeddings**
Use the Embed API to create multimodal embeddings that are contextual vector representations for your videos and texts. Twelve Labs video embeddings capture all the subtle cues and interactions between different modalities, including the visual expressions, body language, spoken words, and the overall context of the video, encapsulating the essence of all these modalities and their interrelations over time.

To create video embeddings, you must first upload your videos, and the platform must finish processing them. Uploading and processing videos require some time. Consequently, creating embeddings is an asynchronous process comprised of three steps:

1. Upload and process a video: When you start uploading a video, the platform creates a video embedding task and returns its unique task identifier.

2. Monitor the status of your video embedding task: Use the unique identifier of your task to check its status periodically until it's completed.

3. Retrieve the embeddings: After the video embedding task is completed, retrieve the video embeddings by providing the task identifier.


In [ ]:
from typing import List

from twelvelabs import TwelveLabs
from twelvelabs.types import VideoSegment
from twelvelabs.embed import TasksStatusResponse

client = TwelveLabs(api_key=TL_API_KEY)

task = client.embed.tasks.create(
    model_name="Marengo-retrieval-2.7",
    video_url="<YOUR_VIDEO_URL>",
    # video_clip_length=5,
    # video_start_offset_sec=30,
    # video_end_offset_sec=60,
    # video_embedding_scope=["clip", "video"]
)
print(f"Created video embedding task: id={task.id}")

def on_task_update(task: TasksStatusResponse):
    print(f"  Status={task.status}")

status = client.embed.tasks.wait_for_done(sleep_interval=5, task_id=task.id, callback=on_task_update)
print(f"Embedding done: {status.status}")

task = client.embed.tasks.retrieve(
    task_id=task.id,
    embedding_option=["visual-text", "audio"]
)

def print_segments(segments: List[VideoSegment], max_elements: int = 5):
    for segment in segments:
        print(f"  embedding_scope={segment.embedding_scope} embedding_option={segment.embedding_option} start_offset_sec={segment.start_offset_sec} end_offset_sec={segment.end_offset_sec}")
        first_few = segment.float_[:max_elements]
        print(
            f"  embeddings: [{', '.join(str(x) for x in first_few)}...] (total: {len(segment.float_)} values)"
        )


if task.video_embedding is not None and task.video_embedding.segments is not None:
    print_segments(task.video_embedding.segments)

